In [1]:
import tensorflow as tf
import keras
import numpy as np
import keras.backend as K

Using TensorFlow backend.


In [2]:
def get_session(number=None):
    config = tf.ConfigProto()
    # config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)
    # return tf.compat.v1.Session(config=config)


# %%
sess = get_session()
K.set_session(sess)

In [3]:
np.random.seed(1)
a = np.random.random(size=12)
a = a.reshape((1,3,2,2))
a

array([[[[4.17022005e-01, 7.20324493e-01],
         [1.14374817e-04, 3.02332573e-01]],

        [[1.46755891e-01, 9.23385948e-02],
         [1.86260211e-01, 3.45560727e-01]],

        [[3.96767474e-01, 5.38816734e-01],
         [4.19194514e-01, 6.85219500e-01]]]])

In [4]:
w = tf.convert_to_tensor(a)

In [9]:
ws=[]
for c in range(10):
    w = tf.convert_to_tensor(a)
    w = tf.reduce_mean(w, axis=-1)
    w = tf.keras.backend.batch_flatten(w)
    w = tf.expand_dims(w, -1)
    ws.append(w)
res = tf.concat(ws, axis=-1)

In [10]:
res.eval(session=sess)

array([[[0.56867325, 0.56867325, 0.56867325, 0.56867325, 0.56867325,
         0.56867325, 0.56867325, 0.56867325, 0.56867325, 0.56867325],
        [0.15122347, 0.15122347, 0.15122347, 0.15122347, 0.15122347,
         0.15122347, 0.15122347, 0.15122347, 0.15122347, 0.15122347],
        [0.11954724, 0.11954724, 0.11954724, 0.11954724, 0.11954724,
         0.11954724, 0.11954724, 0.11954724, 0.11954724, 0.11954724],
        [0.26591047, 0.26591047, 0.26591047, 0.26591047, 0.26591047,
         0.26591047, 0.26591047, 0.26591047, 0.26591047, 0.26591047],
        [0.4677921 , 0.4677921 , 0.4677921 , 0.4677921 , 0.4677921 ,
         0.4677921 , 0.4677921 , 0.4677921 , 0.4677921 , 0.4677921 ],
        [0.55220701, 0.55220701, 0.55220701, 0.55220701, 0.55220701,
         0.55220701, 0.55220701, 0.55220701, 0.55220701, 0.55220701]]])

In [8]:
res.shape

TensorShape([Dimension(1), Dimension(6), Dimension(10)])

In [3]:
np.random.seed(1)
a = np.random.random(size=10)
a = a / np.sum(a)

In [4]:
np.random.seed(1)
b = np.random.random(size=(20,10))

In [5]:
y_pred = tf.expand_dims(tf.convert_to_tensor(a), 0)
W = tf.expand_dims(tf.convert_to_tensor(b), 0)*5

In [6]:
scale=1.0
EPS = 1e-12

In [7]:
A = tf.matrix_diag(y_pred)  # (B, C, C)
p = tf.expand_dims(y_pred, 2)  # (B, C, 1)
A = A - tf.matmul(p, tf.matrix_transpose(p))  # A-pp^T
A = A + tf.eye(int(A.shape[1]), int(A.shape[2]), batch_shape=[1], dtype=tf.float64) * EPS
e, U = tf.linalg.eigh(A)
e = tf.clip_by_value(e, clip_value_min=1e-20, clip_value_max=np.inf)
Sigma = tf.matrix_diag(tf.sqrt(e))
L = tf.matmul(U, Sigma)
B = tf.matmul(W, L)  # (B, K, C) x (B, C, C) --> (B, K, C)
B_T = tf.matrix_transpose(B)  # (B, C, K)
C = tf.matmul(B_T, B)  # (B, C, C)
C = C + tf.eye(int(C.shape[1]), int(C.shape[2]), batch_shape=[1], dtype=tf.float64) * EPS
H_e, _ = tf.linalg.eigh(C)
ave_beta = tf.reduce_mean(
    H_e[:, -1]
)  # Eigenvalues. Shape is [..., N]. Sorted in non-decreasing order.
print(ave_beta * scale)

Tensor("mul_3:0", shape=(), dtype=float64)


In [8]:
ave_beta.eval(session=sess)

12.469716966787345

In [12]:
e.eval(session=sess)

array([[9.99974053e-13, 4.03892524e-05, 3.15778377e-02, 4.98125842e-02,
        6.54927495e-02, 1.00421858e-01, 1.15814323e-01, 1.29397456e-01,
        1.56595191e-01, 2.07795432e-01]])

In [14]:
U.eval(session=sess)

array([[[ 3.16227766e-01, -1.05385725e-01, -9.32920391e-02,
          9.65895487e-02, -1.77445340e-01,  1.58021695e-01,
         -3.26478403e-01,  7.32668908e-01, -3.87161457e-01,
          1.45318026e-01],
        [ 3.16227766e-01, -1.05372201e-01, -8.24359955e-02,
          7.70544871e-02, -1.25733863e-01,  6.82200116e-02,
         -8.33936546e-02,  4.00012458e-02,  2.22315053e-01,
         -8.93152872e-01],
        [ 3.16227766e-01,  9.48683292e-01,  8.19048034e-05,
         -4.40300902e-05,  4.98528829e-05, -1.38682129e-05,
          1.29385416e-05, -4.88762216e-06, -1.63126334e-05,
          1.44359768e-05],
        [ 3.16227766e-01, -1.05397913e-01, -1.05850561e-01,
          1.25181831e-01, -2.81897174e-01, -8.49845149e-01,
          2.00770805e-01, -5.01806453e-02, -1.11577232e-01,
          7.09729625e-02],
        [ 3.16227766e-01, -1.05444917e-01, -2.20014954e-01,
         -8.87523841e-01,  2.22139812e-01, -3.32164829e-02,
          2.77879799e-02, -9.80348874e-03, -2.980348